In [18]:
#session 3 que 2
import numpy as np
from scipy.interpolate import CubicSpline

shocks = {
    506: 0.486,
    258: 0.661,
    358: 0.371,
    735: 0.293,
    166: 0.203,
    781: 0.633,
    789: 0.529,
    822: 0.86,
    728: 0.038,
    725: 0.886
}

pnl_grid = {
    506: ([0.465,0.05,0.345,0.629,0.289,0.05,0.243,0.822,0.665,0.856], [-220.983,-217.841,-220.074,-222.224,-219.65,-217.841,np.nan,-223.685,np.nan,-223.942]),
    258: ([0.473,0.232,0.649,0.19,0.962,0.93,0.639,0.059,0.831,0.837], [-653.225,-654.91,-651.994,-655.203,-649.806,-650.03,-652.064,np.nan,-650.722,-650.68]),
    358: ([0.97,0.836,0.031,0.831,0.634,0.56,0.046,0.094,0.202,0.198], [687.434,686.908,683.749,686.888,686.115,685.825,683.808,683.997,684.42,684.405]),
    735: ([0.949,0.326,0.205,0.952,0.543,0.032,0.926,0.826,0.875,0.846], [-426.147,-428.488,-428.943,-426.136,-427.673,-429.593,-426.234,-426.609,np.nan,-426.534]),
    166: ([0.041,0.48,0.575,0.09,0.412,0.12,0.584,0.306,0.981,0.649], [769.374,762.653,761.199,768.624,763.694,np.nan,761.061,np.nan,754.983,760.066]),
    781: ([0.569,0.334,0.102,0.744,0.685,0.546,0.85,0.097,0.791,0.249], [-449.069,-448.246,np.nan,-449.682,np.nan,-448.989,-450.054,-447.416,-449.847,-447.948]),
    789: ([0.864,0.536,0.223,0.578,0.646,0.147,0.401,0.535,0.51,0.69], [477.718,472.913,468.328,473.528,474.525,467.215,470.936,472.899,472.532,475.169]),
    822: ([0.051,0.068,0.386,0.224,0.618,0.969,0.581,0.616,0.405,0.573], [-999.429,-999.695,-1004.679,-1002.14,-1008.315,-1013.816,-1007.735,-1008.284,-1004.977,-1007.61]),
    728: ([0.605,0.18,0.575,0.316,0.723,0.911,0.98,0.291,0.823,0.63], [336.468,332.996,336.223,334.107,np.nan,338.968,339.531,333.903,338.249,336.672]),
    725: ([0.76,0.703,0.223,0.785,0.211,0.48,0.644,0.551,0.871,0.275], [-204.815,-205.658,-212.755,np.nan,-212.933,-208.955,-206.53,-207.905,-203.174,-211.986]),
}


interpolated_pnl = {}

for issuer_id, shock_value in shocks.items():
    if issuer_id in pnl_grid:
        shock_list, pnl_list = pnl_grid[issuer_id]
        
        # Filter out np.nan shocks and corresponding pnl values
        valid_indices = np.isfinite(shock_list) & np.isfinite(pnl_list)
        shock_list = np.array(shock_list)[valid_indices]
        pnl_list = np.array(pnl_list)[valid_indices]
        
        # Sort the shock_list and pnl_list in increasing order of shocks
        sorted_indices = np.argsort(shock_list)
        shock_list = shock_list[sorted_indices]
        pnl_list = pnl_list[sorted_indices]
        
        # Create a CubicSpline object
        try:
            cs = CubicSpline(shock_list, pnl_list)
            
            # Interpolate pnl using the given shock_value
            interpolated_pnl[issuer_id] = cs(shock_value)
        except ValueError:
            print(f"Error interpolating for issuer {issuer_id}: x values are not strictly increasing")

print(interpolated_pnl)



Error interpolating for issuer 506: x values are not strictly increasing
{258: array(-651.91003419), 358: array(685.07423632), 735: array(-428.6120454), 166: array(766.89413785), 781: array(-449.29245819), 789: array(472.81350498), 822: array(-1011.89993308), 728: array(331.82333689), 725: array(-202.95258113)}


In [22]:
#session 3 que 3


cutoffs = {
    506: 0.4,
    258: 0.6,
    358: 0.3,
    735: 0.3,
    166: 0.2,
    781: 0.6,
    789: 0.5,
    822: 0.8,
    728: 0.4,
    725: 0.4
}

# Simulation parameters
num_simulations = 1000
num_timesteps = 4

# Function to calculate default pnl
# Function to calculate default pnl
def calculate_default_pnl(shock_value, issuer_id):
    if issuer_id in pnl_grid:
        shock_list, pnl_list = pnl_grid[issuer_id]
        valid_indices = np.isfinite(shock_list) & np.isfinite(pnl_list)
        unique_shock_list = np.unique(np.array(shock_list)[valid_indices])
        interpolated_pnls = []

        for unique_shock in unique_shock_list:
            shock_indices = np.where(np.array(shock_list) == unique_shock)[0]
            corresponding_pnls = np.array(pnl_list)[shock_indices]
            default_pnl = np.max(corresponding_pnls)
            interpolated_pnls.append(default_pnl)

        interpolated_pnls = np.array(interpolated_pnls)
        
        # Create a CubicSpline object
        cs = CubicSpline(unique_shock_list, interpolated_pnls)
        
        # Calculate interpolated pnl using the given shock_value
        interpolated_pnl = cs(shock_value)
        
        return interpolated_pnl
    else:
        return 0.0


# Perform the simulation
total_default_pnl = {issuer_id: 0.0 for issuer_id in pnl_grid.keys()}

for _ in range(num_simulations):
    for issuer_id, cutoff in cutoffs.items():
        shock_values = np.random.RandomState(issuer_id).rand(num_timesteps)
        defaulted = False
        for i, shock in enumerate(shock_values):
            if defaulted:
                total_default_pnl[issuer_id] += 0.0
            elif shock > cutoff:
                defaulted = True
                default_pnl = calculate_default_pnl(shock, issuer_id)
                total_default_pnl[issuer_id] += default_pnl
            else:
                total_default_pnl[issuer_id] += 0.0

# Calculate expected default pnl for each issuer
expected_default_pnl = {issuer_id: total_pnl / num_simulations for issuer_id, total_pnl in total_default_pnl.items()}

# Print the results
for issuer_id, pnl in expected_default_pnl.items():
    print(f"Issuer {issuer_id}: Expected Default PnL = {pnl}")


Issuer 506: Expected Default PnL = -223.8807040748086
Issuer 258: Expected Default PnL = -651.9314837485578
Issuer 358: Expected Default PnL = 686.4253637569135
Issuer 735: Expected Default PnL = -427.1661691326838
Issuer 166: Expected Default PnL = 765.4436012893289
Issuer 781: Expected Default PnL = -449.52580326457877
Issuer 789: Expected Default PnL = 476.6928857514708
Issuer 822: Expected Default PnL = -1013.0063547659722
Issuer 728: Expected Default PnL = 338.35111535140413
Issuer 725: Expected Default PnL = -208.72646933576655
